# 🎯 Customer Segmentation Using K-Means (Unsupervised Learning)

**Industry-Grade Customer Behavior Analysis with Transactional Data**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RansiluRanasinghe/Customer-Segmentation-K-Means/blob/main/notebook.ipynb)

---

## 📌 Project Overview

This notebook implements an **end-to-end, production-aligned unsupervised machine learning pipeline** for customer segmentation using **K-Means clustering**.

The project simulates how machine learning engineers and data science teams analyze large-scale retail transaction data to discover meaningful customer segments **without labeled data**. The focus is on **data validation, feature engineering, model robustness, and business interpretability**, rather than on predictive accuracy.

---

## 🎯 What This Notebook Does

Using real-world online retail transaction data, the pipeline:

1. ✅ **Cleans and validates** raw transactional records
2. ✅ **Aggregates data** at the customer level
3. ✅ **Engineers RFM** (Recency, Frequency, Monetary) behavioral features
4. ✅ **Applies K-Means clustering** to identify natural customer groups
5. ✅ **Evaluates clusters** using industry-appropriate unsupervised metrics
6. ✅ **Interprets results** from a business and operational perspective

---

## 💡 Design Philosophy

This notebook reflects **industry best practices** for unsupervised modeling and is designed to be:
- 📊 **Reproducible** — Clear, sequential workflow
- 🔍 **Interpretable** — Business-focused insights
- 🔧 **Extendable** — Ready for real-world customer analytics use cases

---

**Author:** Ransilu Ranasinghe  
**GitHub:** [RansiluRanasinghe](https://github.com/RansiluRanasinghe)  
**LinkedIn:** [ransilu-ranasinghe](https://www.linkedin.com/in/ransilu-ranasinghe-a596792ba)

---

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [2]:
plt.style.use("seaborn-v0_8-darkgrid")

import warnings
warnings.filterwarnings("ignore")

In [3]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

####Loading the data

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mashlyn/online-retail-ii-uci")

print("Path to dataset files:", path)

100%|██████████| 14.5M/14.5M [00:00<00:00, 126MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mashlyn/online-retail-ii-uci/versions/3


In [5]:
import os

DATASET_PATH = "/root/.cache/kagglehub/datasets/mashlyn/online-retail-ii-uci/versions/3"

os.listdir(DATASET_PATH)

['online_retail_II.csv']

In [6]:
df = pd.read_csv(os.path.join(DATASET_PATH, "online_retail_II.csv"), encoding="ISO-8859-1")

display(df.head(5))

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [7]:
print("Dataset shape:", df.shape)
print("Column names:", list(df.columns))
print("\nDatatypes: \n\n", df.dtypes)

Dataset shape: (1067371, 8)
Column names: ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']

Datatypes: 

 Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
dtype: object


####Dataset Analysis

In [8]:
column_mapping = {
    "Invoice" : "InvoiceNO",
    "Price" : "UnitPrice",
    "Customer ID" : "CustomerID"
}

df = df.rename(columns=column_mapping)

display(df.head(5))

,InvoiceNO,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [9]:
required_columns = ["InvoiceNO", "Quantity", "UnitPrice", "InvoiceDate", "CustomerID"]
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
  raise ValueError(f"Missing required columns: {missing_columns}")

In [10]:
print("Record count: ", len(df))
print("\n Missing values per column: ")
print(df.isna().sum())

Record count:  1067371

 Missing values per column: 
InvoiceNO           0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     243007
Country             0
dtype: int64


In [11]:
print("Checking Bussiness rule violations: ")

print("Negative quantities: ", (df["Quantity"] <= 0).sum())
print("Negative prices: ", (df["UnitPrice"] <= 0).sum())

print("Missing customer ID ", df["Quantity"].isnull().sum())

Checking Bussiness rule violations: 
Negative quantities:  22950
Negative prices:  6207
Missing customer ID  0


In [12]:
cancellations = df[df["InvoiceNO"].astype(str).str.startswith("C")]
print("Cancellations: ", len(cancellations))

Cancellations:  19494


In [13]:
print("Unique Products: ", df["StockCode"].nunique())
print("Unique Customers: ", df["CustomerID"].nunique())
print("Countries represented: ", df["Country"].nunique())

Unique Products:  5305
Unique Customers:  5942
Countries represented:  43


####Data Cleaning

In [14]:
df_clean = df.copy()

In [15]:
df_clean["InvoiceDate"] = pd.to_datetime(df_clean["InvoiceDate"])

In [16]:
df_clean = df_clean[~df_clean['InvoiceNO'].astype(str).str.startswith('C', na=False)]

In [17]:
df_clean = df_clean[df_clean['Quantity'] > 0]
df_clean = df_clean[df_clean['UnitPrice'] > 0]

In [18]:
initial_count = len(df_clean)

df_clean = df_clean.dropna(subset=['CustomerID'])
removed_count = initial_count - len(df_clean)
print(f"Removed {removed_count} rows with missing CustomerID.")

Removed 236121 rows with missing CustomerID.


In [19]:
df_clean['CustomerID'] = df_clean['CustomerID'].astype(int)

In [20]:
initial_count = len(df)

df_clean = df_clean.drop_duplicates()
removed_count = initial_count - len(df_clean)
print(f"Removed {removed_count} duplicate rows.")

Removed 287946 duplicate rows.


In [21]:
df_clean["TotalValue"] = df_clean["Quantity"] * df_clean["UnitPrice"]

In [22]:
print(f"\nDate range: {df_clean['InvoiceDate'].min()} to {df_clean['InvoiceDate'].max()}")
print(f"Cleaned dataset shape: {df_clean.shape}")


Date range: 2009-12-01 07:45:00 to 2011-12-09 12:50:00
Cleaned dataset shape: (779425, 9)


####Explotary Data Analysis(EDA)

In [23]:
print("Transaction Statics: \n")

print("Average Quantity: ", df_clean["Quantity"].mean())
print("Mediun Quantity: ", df_clean["Quantity"].median())
print("95th percentile quantity: ", df_clean["Quantity"].quantile(0.95), "\n")

print("Average UnitPrice: ", df_clean["UnitPrice"].mean())
print("Mediun UnitPrice: ", df_clean["UnitPrice"].median())
print("95th percentile UnitPrice: ", df_clean["UnitPrice"].quantile(0.95), "\n")

print("Average Transaction value: ", df_clean["TotalValue"].mean())
print("Mediun Transaction value: ", df_clean["TotalValue"].median())
print("95th percentile Transaction value: ", df_clean["TotalValue"].quantile(0.95), "\n")

Transaction Statics: 

Average Quantity:  13.489369727683869
Mediun Quantity:  6.0
95th percentile quantity:  36.0 

Average UnitPrice:  3.2184879853738333
Mediun UnitPrice:  1.95
95th percentile UnitPrice:  8.5 

Average Transaction value:  22.29182316194631
Mediun Transaction value:  12.48
95th percentile Transaction value:  67.5 



In [24]:
customer_stats = df_clean.groupby("CustomerID").agg({
    "InvoiceNO": "nunique",
    "TotalValue": "sum"
}).rename(columns={'InvoiceNO': 'TransactionCount', 'TotalValue': 'TotalSpend'})

In [25]:
print("Customer Level Statistics: \n")

print("Total Customers: ", len(customer_stats))

print("Average Transactions per customer: ", customer_stats["TransactionCount"].mean())
print("Mediun Transactions per customer: ", customer_stats["TransactionCount"].median(), "\n")

print("Average Spend per customer: ", customer_stats["TotalSpend"].mean())
print("Mediun Spend per customer: ", customer_stats["TotalSpend"].median(), "\n")

Customer Level Statistics: 

Total Customers:  5878
Average Transactions per customer:  6.289384144266758
Mediun Transactions per customer:  3.0 

Average Spend per customer:  2955.9040945899965
Mediun Spend per customer:  867.74 



In [26]:
top_20_percent_customers = int(len(customer_stats) * 0.2)
top_customers = customer_stats.nlargest(top_20_percent_customers, 'TotalSpend')
top_customers_share = top_customers['TotalSpend'].sum() / customer_stats['TotalSpend'].sum()

print("Pareto Analysis: \n")
print(f"Top 20% customers represent {top_customers_share:.1%} of total revenue.")

Pareto Analysis: 

Top 20% customers represent 77.2% of total revenue.


In [27]:
print("Outlier assesment with Unique IQR: \n")

for column in ['Quantity', 'UnitPrice', 'TotalValue']:
  Q1 = df_clean[column].quantile(0.25)
  Q3 = df_clean[column].quantile(0.75)

  IQR = Q3 -Q1

  upper_bound = Q3 + 1.5 * IQR

  outliers = df_clean[df_clean[column] > upper_bound]
  print(f"{column}: {len(outliers):,} outliers ({len(outliers)/len(df_clean):.1%} of transactions)")

Outlier assesment with Unique IQR: 

Quantity: 51,119 outliers (6.6% of transactions)
UnitPrice: 65,463 outliers (8.4% of transactions)
TotalValue: 63,562 outliers (8.2% of transactions)


####Feature Engineering - RFM Modeling

In [29]:
ref_date = df_clean["InvoiceDate"].max() + pd.Timedelta(days=1)

In [32]:
rfm_data = df_clean.groupby("CustomerID").agg({
    "InvoiceDate" : lambda x: (ref_date - x.max()).days,
    "InvoiceNO" : "nunique",
    "TotalValue" : "sum"
}).rename(columns={
    "InvoiceDate" : "Recency",
    "InvoiceNO" : "Frequency",
    "TotalValue" : "Monetary"
})

In [33]:
print("RFM Data Statistics: \n")

print(rfm_data.describe())

RFM Data Statistics: 

           Recency    Frequency       Monetary
count  5878.000000  5878.000000    5878.000000
mean    201.331916     6.289384    2955.904095
std     209.338707    13.009406   14440.852688
min       1.000000     1.000000       2.950000
25%      26.000000     1.000000     342.280000
50%      96.000000     3.000000     867.740000
75%     380.000000     7.000000    2248.305000
max     739.000000   398.000000  580987.040000


In [34]:
print("Customers with zero frquency", (rfm_data["Frequency"] == 0).sum())
print("Customers with zero monetary", (rfm_data["Monetary"] == 0).sum())

Customers with zero frquency 0
Customers with zero monetary 0


In [35]:
rfm_data = rfm_data[(rfm_data["Frequency"] > 0) & (rfm_data["Monetary"] > 0)]

print("Final RFM dataset shape: ", rfm_data.shape)

Final RFM dataset shape:  (5878, 3)
